In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8824,2024-04-08,Suécia Basketligan,14:04,Nassjo,BC Lulea,2.03,1.74,175.5,1.87,1.79,1.5,1.88,1.60,bqrgcRw1,0.492611,0.574713,0.534759,0.558659,0.067323,188.030,93.174142,0.495528,1.8,1.643168,0.912871,241.98,2.162,1.188221,0.549594,34.0,197.554,108.615324,0.549801,1.8,1.643168,0.912871,118.80,2.198,1.216150,0.553298,24.0,74,90,3.27,1.32,184.486,195.310,0.108786,0.030912,0.113787,-2.53,-0.506,-2.035573,0.598366,0.5,-0.098366,3.03,0.606,1.221122,0.624261,0.7,0.075739
8825,2024-04-08,Lituânia Nkl,13:00,Jurbarkas,BC Olimpas Palanga,2.50,1.49,165.5,1.85,1.85,0.0,0.00,0.00,jVhkr3K1,0.400000,0.671141,0.540541,0.540541,0.071141,198.822,67.393118,0.338962,1.2,1.643168,1.369306,204.75,2.320,0.823954,0.355153,-30.0,125.472,17.989428,0.143374,2.4,1.341641,0.559017,120.52,1.330,0.192873,0.145017,82.0,63,92,3.25,1.31,154.932,139.172,0.357984,0.000000,NaN,-0.62,-0.124,-12.096774,0.542645,0.5,-0.042645,0.38,0.076,6.447368,0.692302,0.9,0.207698
8826,2024-04-08,Lituânia Lkl,12:50,Jonava,Neptunas,2.32,1.56,169.5,1.87,1.79,0.0,0.00,0.00,CMWZVDrl,0.431034,0.641026,0.534759,0.558659,0.072060,236.132,81.708590,0.346029,1.2,1.643168,1.369306,131.22,2.954,1.059306,0.358601,-17.0,217.008,123.500659,0.569106,3.0,0.000000,0.000000,119.21,2.528,1.495734,0.591667,27.0,81,91,1.62,1.31,246.214,236.896,0.277011,0.030912,NaN,-0.18,-0.036,-36.666667,0.000000,0.0,0.000000,4.88,0.976,0.573770,0.455733,0.6,0.144267
8827,2024-04-08,Israel Wbl Feminina,12:00,Elitzur Ramla F,Hapoel Jerusalem F,1.35,3.02,151.5,1.84,1.84,0.0,0.00,0.00,Cpu6mUmS,0.740741,0.331126,0.543478,0.543478,0.071867,98.578,6.374913,0.064669,2.4,1.341641,0.559017,97.68,1.176,0.094499,0.080356,59.0,135.240,38.457395,0.284364,2.4,1.341641,0.559017,198.90,1.690,0.503835,0.298127,29.0,88,85,1.11,2.34,95.686,166.322,0.540443,0.000000,NaN,-0.73,-0.146,-2.397260,0.000000,0.0,0.000000,3.55,0.710,2.845070,0.000000,0.0,0.000000
8828,2024-04-08,Japão Liga Feminina W,07:00,Fujitsu F,Chanson V-Magic F,1.40,2.80,141.5,1.85,1.85,0.0,0.00,0.00,EDBPCJlf,0.714286,0.357143,0.540541,0.540541,0.071429,114.018,18.379081,0.161195,2.4,1.341641,0.559017,98.70,1.644,0.259191,0.157659,31.0,171.088,73.075660,0.427123,1.8,1.643168,0.912871,195.96,2.328,0.787572,0.338304,-8.0,70,71,1.41,2.76,128.026,174.504,0.471405,0.000000,NaN,0.12,0.024,16.666667,0.000000,0.0,0.000000,1.00,0.200,9.000000,0.571078,0.5,-0.071078
8829,2024-04-08,Itália Série A2,15:00,Basket Torino,Rimini,1.37,2.91,162.5,1.86,1.80,0.0,0.00,0.00,jqGmqbPB,0.729927,0.343643,0.537634,0.555556,0.073570,141.028,28.613231,0.202890,1.8,1.643168,0.912871,127.17,1.770,0.452548,0.255677,25.0,164.958,107.670252,0.652713,3.0,0.000000,0.000000,114.39,2.034,1.319045,0.648498,52.0,81,93,1.57,1.23,166.966,258.326,0.508853,0.023184,NaN,0.45,0.090,4.111111,0.681570,0.8,0.118430,7.92,1.584,1.205808,0.342131,0.5,0.157869
8830,2024-04-08,Suécia Basketligan,14:04,Sodertalje,Borås,2.85,1.40,166.5,1.88,1.85,0.0,0.00,0.00,CSus0Tgr,0.350877,0.714286,0.531915,0.540541,0.065163,292.556,179.980615,0.615201,1.2,1.643168,1.369306,296.01,3.648,2.176791,0.596708,-29.0,112.742,11.613368,0.103008,2.4,1.341641,0.559017,111.60,1.324,0.258612,0.195326,32.0,69,93,4.29,1.20,225.084,143.484,0.482496,0.011374,NaN,-2.03,-0.406,-4.556650,0.544926,0.5,-0.044926,1.97,0.394,1.015228,0.644072,0.8,0.15592

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8859,13:30,Dinamarca Liga De Basquete,Randers,Næstved,1.82,Back Home
